In [1]:
# 역전파 자동화

In [1]:
class Variable:
    def __init__(self, data):
        self.data = data # 0.5
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator # 이부분이 None이 될 때까지  variable 의 creator이다.
        if f is not None: # creator 가 없는 경우까지 즉 x는 creator가 없다.
            x = f.input # x 는 Variable이다.
            x.grad = f.backward(self.grad) # 이전의 grad값을 가져와 backward 시킨다. -> x.grad가 나옴 재귀 호출이기 때문에 self.grad 를 쓴다.
            x.backward() # 다시 backward 메소드 호출한다. # 안에서 또 x.backward() 재귀호출

In [2]:
class Function:
    def __call__(self, input):
        x = input.data # variable 클래스의 데이터를 끄집어 낸다.
        y = self.forward(x) # forward 시켜 새로운 데이터를 만든다.
        output = Variable(y) # forward 시킨 출력값

        output.set_creator(self) # 출력 변수에 창조자를 설정한다.( 어떤 클래스에서 생성되었는가?
        print(output.creator)
        self.input = input # self 로 두면 다른 메서드에서 사용할 수 있음 , 추후 backward에서 사용할 수 있게 해야함
        self.ouput = output # 출력도 저장한다.
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise  NotImplementedError()

In [3]:
import numpy as np

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        print(x)
        gx = np.exp(x) * gy
        return gx

In [4]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        print(x)
        gx = 2 * x * gy #
        return gx

In [5]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))

In [6]:
a = A(x)
b = B(a)
y = C(b)


In [7]:
assert y.creator == C # 역전파 알고리즘
assert y.creator.input == b 
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [8]:
y.grad = np.array(1.0)

C = y.creator # 함수를 가져온다
b = C.input # 함수의 입력을 가져온다.
b.grad = C.backward(y.grad) # 함수의 backward 메서드를 호출한다.
print(b.grad)

1.2840254166877414
2.568050833375483


In [9]:
B = b.creator
a = B.input
a.grad = B.backward(b.grad)
print(a.grad)

0.25
3.297442541400256


In [10]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

0.5
3.297442541400256


In [11]:
y.grad = np.array(1.0)
y.backward()
print(x.grad)

1.2840254166877414
0.25
0.5
3.297442541400256


In [13]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파

y.grad = np.array(1.0)
y.backward()
print(x.grad)

1.2840254166877414
0.25
0.5
3.297442541400256
